In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import sys
import glob
import pandas as pd
import os
import seaborn as sns

from tqdm import tqdm
from statsmodels.distributions.empirical_distribution import ECDF
from collections import defaultdict
import pickle
import re
import json
from pathlib import Path
import scipy.stats
import time

from open_spiel.python.algorithms.exploitability import nash_conv, best_response
from open_spiel.python.examples.ubc_plotting_utils import *
from open_spiel.python.examples.ubc_sample_game_tree import sample_game_tree, flatten_trees, flatten_tree
from open_spiel.python.examples.ubc_clusters import projectPCA, fitGMM
from open_spiel.python.examples.ubc_utils import *
import open_spiel.python.examples.ubc_dispatch as dispatch
from open_spiel.python.visualizations import ubc_treeviz


from auctions.webutils import *

os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

from open_spiel.python.examples.ubc_cma import *

output_notebook()
from open_spiel.python.games.clock_auction_base import InformationPolicy, ActivityPolicy, UndersellPolicy, TiebreakingPolicy
from open_spiel.python.algorithms.exploitability import nash_conv, best_response
from open_spiel.python.examples.ubc_decorators import TakeSingleActionDecorator, TremblingAgentDecorator, ModalAgentDecorator

Loading BokehJS ...

In [2]:
name = 'oct4_encumberedheuristic_oct4_encumberedheuristic_3_base_undersell_allowed-cfr_port_7_extexternal_plus_full-101'
run = EquilibriumSolverRun.objects.get(name=name, experiment__name='oct13_external_heuristics_v1')
game, final_checkpoint, policy = get_results(run, load_policy=True)
evaluation = final_checkpoint.get_modal_eval()

In [3]:
%%time
retval = nash_conv(game, policy, return_only_nash_conv=False, restrict_to_heuristics=True)
print(retval)


KeyboardInterrupt



In [7]:
%%time
env_and_policy = make_env_and_policy(game, dict(final_checkpoint.equilibrium_solver_run.config))
for agent in env_and_policy.agents:
    agent.policy = policy
for player in range(game.num_players()):
    env_and_policy.agents[player] = ModalAgentDecorator(env_and_policy.agents[player])
modal_policy = env_and_policy.make_policy()
modal_retval = nash_conv(game, modal_policy, return_only_nash_conv=False, restrict_to_heuristics=True)
print(modal_retval)

_NashConvReturn(nash_conv=20.24767626552017, player_improvements=array([18.6684278 ,  1.57924847]), br_policies=[<open_spiel.python.algorithms.best_response.BestResponsePolicy object at 0x7fb020bc98e0>, <open_spiel.python.algorithms.best_response.BestResponsePolicy object at 0x7fb020bc9940>])
CPU times: user 842 ms, sys: 24.1 ms, total: 866 ms
Wall time: 865 ms


In [77]:
def draw_game_tree(game, policy, br_policies=None, fname='treeviz.png', figure_dir='figures/2023-06-27-graphviz'):
    node_policy_decorator, edge_policy_decorator = ubc_treeviz.make_policy_decorators(policy, br_policies)
    gametree = ubc_treeviz.GameTree(
        game,
        node_decorator=node_policy_decorator,
        edge_decorator=edge_policy_decorator,
        group_infosets=False,
        group_terminal=False,
        group_pubsets=False, 
        target_pubset='*',
        depth_limit=20,
        # state_prob_limit=0.001,
        action_prob_limit=0.14, 
        policy=policy,
        br_policies=br_policies,
    )
    
    outfile = os.path.join(figure_dir, fname)
    gametree.draw(outfile, prog='dot')
    print("Game tree saved to file", outfile)
    


In [36]:
draw_game_tree(game, policy, fname='mixing.pdf', figure_dir='figures')

Game tree saved to file figures/mixing.pdf


In [50]:
draw_game_tree(game, policy, retval.br_policies, fname='mixing_br.pdf', figure_dir='figures')

Game tree saved to file figures/mixing_br.pdf


In [37]:
draw_game_tree(game, modal_policy, fname='mixing_modal.pdf', figure_dir='figures')

Game tree saved to file figures/mixing_modal.pdf


In [38]:
draw_game_tree(game, modal_policy, modal_retval.br_policies, fname='mixing_modal_br.pdf', figure_dir='figures')

Game tree saved to file figures/mixing_modal_br.pdf


In [154]:
env_and_policy = make_env_and_policy(game, dict(final_checkpoint.equilibrium_solver_run.config))
# for agent in env_and_policy.agents:
#     agent.policy = policy

In [221]:
class BRAgent:

    def __init__(self, i, policy):
        self.player_id = i
        self.policy = policy

    def step(self, time_step, is_evaluation=False):
        if isinstance(time_step, list):
            return [self._act(ts) for ts in time_step]
        else:
            return self._act(time_step)

    def _act(self, time_step):
        if time_step.last():
            return

        state = time_step.observations['state']
        # print(time_step)
        action_probs = {a: 0.0 for a in time_step.observations['legal_actions'][self.player_id]}
        action_probs = {**action_probs, **self.policy.action_probabilities(state)}
        # action_probs = defaultdict(float, {a:action_probs[a] for a in action_probs})
        action = fast_choice(list(action_probs.keys()), list(action_probs.values())) # TODO: Give rng
        probs = np.array(list(action_probs.values())) # TODO: is this indexed right? It's good enough to compute entropies...
        # print(probs)
        return StepOutput(action=action, probs=probs)

In [222]:
env_and_policy = make_env_and_policy(game, dict(final_checkpoint.equilibrium_solver_run.config))

env_and_policy.agents[0] = BRAgent(0, modal_retval.br_policies[0])
env_and_policy.agents[1].policy = policy
env_and_policy.agents[1] = ModalAgentDecorator(env_and_policy.agents[1])
modal_br_policy_0 = env_and_policy.make_policy()
draw_game_tree(game, modal_br_policy_0, fname='mixing_modal_br0.pdf', figure_dir='figures')

Game tree saved to file figures/mixing_modal_br0.pdf


In [217]:
modal_br_policy_0._agents[0].

SyntaxError: invalid syntax (804620098.py, line 1)

In [49]:
game.auction_params

AuctionParams(opening_prices=[32, 19, 13], licenses=array([1, 1, 1]), license_names=['Ontario', 'Quebec'], activity=[32, 19, 13], num_products=3, increment=0.3, reveal_type_round=-1, skip_single_chance_nodes=True, max_round=10, player_types=defaultdict(<class 'list'>, {0: [{'prob': 0.5, 'bidder': <open_spiel.python.games.clock_auction_bidders.EnumeratedValueBidder object at 0x7f9b511f8640>}, {'prob': 0.5, 'bidder': <open_spiel.python.games.clock_auction_bidders.EnumeratedValueBidder object at 0x7f9b511f8970>}], 1: [{'prob': 0.5, 'bidder': <open_spiel.python.games.clock_auction_bidders.EnumeratedValueBidder object at 0x7f9b511f8c70>}, {'prob': 0.5, 'bidder': <open_spiel.python.games.clock_auction_bidders.EnumeratedValueBidder object at 0x7f9b511f88b0>}]}), all_bids=array([[0, 0, 0],
       [0, 0, 1],
       [0, 1, 0],
       [0, 1, 1],
       [1, 0, 0],
       [1, 0, 1],
       [1, 1, 0],
       [1, 1, 1]]), bid_to_index={(0, 0, 0): 0, (0, 0, 1): 1, (0, 1, 0): 2, (0, 1, 1): 3, (1, 0, 0)

In [48]:
bidder = game.auction_params.player_types[0][1]['bidder']
bidder.values

array([  0,  12,  15,  22, 126, 138, 141, 148])

In [105]:
state = game.new_initial_state().child(1).child(0)

In [106]:
br = modal_retval.br_policies[0]

In [107]:
br.best_response_action(state.information_state_string())

7

In [83]:
br.infosets[state.information_state_string()]

[(Round: 1
  Player 0: regional
  Player 1: regional,
  0.25),
 (Round: 1
  Player 0: regional
  Player 1: regional,
  0.25)]

In [147]:
state = game.new_initial_state().child(1).child(1)

In [148]:
retval.br_policies[0].q_value(state, 4)

0.21258157399704242

In [149]:
retval.br_policies[0].q_value(state, 7)

1.1918572017329945

In [150]:
modal_retval.br_policies[0].q_value(state, 4)

81.4

In [151]:
modal_retval.br_policies[0].q_value(state, 7)

69.82384615384615

In [152]:
modal_retval.br_policies[0].q_value(state.child(4).child(7), 3)

81.4

In [153]:
modal_retval.br_policies[0].best_response_action(state.child(4).child(7).information_state_string())

3

In [137]:
policy.action_probabilities(state.child(4))

{0: 0.00020142419021640362, 7: 0.9997985758097836}

In [56]:
retval.br_policies[0].action_probabilities(state)

{4: 1}

In [57]:
modal_retval.br_policies[0].action_probabilities(state)

{7: 1}

In [28]:
policy._infostates['p1t0 r2 posted[[32.0, 19.0, 13.0], [32.0, 24.7, 16.9]] sub[[0, 0, 0], [1, 1, 1]] a64 agg[[0, 0, 0], [1, 2, 2]] proc[[0, 0, 0], [1, 1, 1]]']

[array([ 0.        ,  0.        , 28.23021104]),
 array([1.00000000e-06, 2.32500575e+03, 2.88499425e+03]),
 7899,
 0.0]

In [229]:
policy._infostates['p0t1 r3 posted[[32.0, 19.0, 13.0], [41.6, 24.7, 16.9], [54.08, 24.7, 16.9]] sub[[0, 0, 0], [1, 1, 1], [1, 0, 0]] a32 agg[[0, 0, 0], [2, 2, 2], [2, 1, 1]] proc[[0, 0, 0], [1, 1, 1], [1, 0, 0]]']

[array([   0.        ,  222.74227128, 1538.08980524]),
 array([  0.54638362,  17.00922613, 125.44439324]),
 3144,
 0.0]

In [230]:
policy._infostates['p0t1 r3 posted[[32.0, 19.0, 13.0], [41.6, 24.7, 16.9], [54.08, 24.7, 16.9]] sub[[0, 0, 0], [1, 1, 1], [1, 0, 0]] a32 agg[[0, 0, 0], [2, 2, 2], [2, 1, 1]] proc[[0, 0, 0], [1, 1, 1], [1, 0, 0]]']

[array([   0.        ,  222.74227128, 1538.08980524]),
 array([  0.54638362,  17.00922613, 125.44439324]),
 3144,
 0.0]

In [228]:
policy._infostates['p1t1 r3 posted[[32.0, 19.0, 13.0], [41.6, 24.7, 16.9], [54.08, 24.7, 16.9]] sub[[0, 0, 0], [1, 1, 1], [1, 1, 1]] a64 agg[[0, 0, 0], [2, 2, 2], [2, 1, 1]] proc[[0, 0, 0], [1, 1, 1], [1, 1, 1]]']

[array([  0.        ,   0.        , 171.22852131, 213.27940948]),
 array([4.73341858e-01, 4.16126591e+00, 3.26680556e+03, 3.52055983e+03]),
 6860,
 0.0]

In [226]:
policy._infostates['p0t1 r4 posted[[32.0, 19.0, 13.0], [41.6, 24.7, 16.9], [54.08, 24.7, 16.9], [54.08, 32.11, 21.97]] sub[[0, 0, 0], [1, 1, 1], [1, 0, 0], [0, 1, 1]] a32 agg[[0, 0, 0], [2, 2, 2], [2, 1, 1], [1, 2, 2]] proc[[0, 0, 0], [1, 1, 1], [1, 0, 0], [0, 1, 1]]']

[array([   0.        , 1742.32433924,    0.        ]),
 array([ 0.09895434,  0.48343083, 14.41761783]),
 1392,
 0.0]

In [224]:
policy._infostates['p1t1 r4 posted[[32.0, 19.0, 13.0], [41.6, 24.7, 16.9], [54.08, 24.7, 16.9], [54.08, 32.11, 21.97]] sub[[0, 0, 0], [1, 1, 1], [1, 1, 1], [1, 1, 1]] a64 agg[[0, 0, 0], [2, 2, 2], [2, 1, 1], [1, 2, 2]] proc[[0, 0, 0], [1, 1, 1], [1, 1, 1], [1, 1, 1]]']

[array([ 0.        , 40.36701648, 46.59125201]),
 array([ 209.16305937, 1575.53174546, 1794.30519817]),
 3586,
 0.0]

In [225]:
policy._infostates['p0t1 r5 posted[[32.0, 19.0, 13.0], [41.6, 24.7, 16.9], [54.08, 24.7, 16.9], [54.08, 32.11, 21.97], [54.08, 41.74, 28.56]] sub[[0, 0, 0], [1, 1, 1], [1, 0, 0], [0, 1, 1], [0, 1, 1]] a32 agg[[0, 0, 0], [2, 2, 2], [2, 1, 1], [1, 2, 2], [1, 2, 2]] proc[[0, 0, 0], [1, 1, 1], [1, 0, 0], [0, 1, 1], [0, 1, 1]]']

[array([  0.        ,   0.        , 254.28033896]),
 array([1.000000e-06, 1.000000e-06, 1.000001e+00]),
 710,
 0.0]

In [6]:
policy._infostates['p1t1 r5 posted[[32.0, 19.0, 13.0], [41.6, 24.7, 16.9], [54.08, 24.7, 16.9], [54.08, 32.11, 21.97], [54.08, 41.74, 28.56]] sub[[0, 0, 0], [1, 1, 1], [1, 1, 1], [1, 1, 1], [1, 1, 1]] a64 agg[[0, 0, 0], [2, 2, 2], [2, 1, 1], [1, 2, 2], [1, 2, 2]] proc[[0, 0, 0], [1, 1, 1], [1, 1, 1], [1, 1, 1], [1, 1, 1]]']

[array([3.76600428, 1.67259365, 0.        ]),
 array([1194.87219609,  540.12780591,   17.000001  ]),
 1753,
 0.0]

# Nov 16: check last-iterate policies

In [29]:
name = 'oct4_encumberedheuristic_oct4_encumberedheuristic_3_base_undersell_allowed-cfr_port_7_extexternal_plus_full-101'
run = EquilibriumSolverRun.objects.get(name=name, experiment__name='oct13_external_heuristics_v1')
game, final_checkpoint, policy = get_results(run, load_policy=True)
evaluation = final_checkpoint.get_modal_eval()

In [10]:
from open_spiel.python.algorithms.mccfr import REGRET_INDEX, MCCFRSolverBase

class CFRLastIteratePolicyWrapper(object):
    def __init__(self, policy, player_id):
        self.policy = policy
        self.player_id = player_id
        
    def action_probabilities(self, state, player_id=None):
        if player_id is None:
            player_id = state.current_player()
        legal_actions = state.legal_actions()
        info_state_key = state.information_state_string(player_id)
        retrieved_infostate = self.policy._infostates.get(info_state_key, None)
        if retrieved_infostate is None:
            return {a: 1 / len(legal_actions) for a in legal_actions}
        else:
            regrets = retrieved_infostate[REGRET_INDEX]
            action_probs = MCCFRSolverBase._regret_matching(None, regrets, len(legal_actions))
            return {a: action_probs[i] for i, a in enumerate(legal_actions)}
            
# test it
env_and_policy = make_env_and_policy(game, dict(final_checkpoint.equilibrium_solver_run.config))
for agent in env_and_policy.agents:
    agent.policy = policy
    
last_iterate_policy = CFRLastIteratePolicyWrapper(env_and_policy.agents[0].policy, 0)
state = game.new_initial_state().child(0).child(0)
display(policy.action_probabilities(state))
display(last_iterate_policy.action_probabilities(state))

{0: 1.864280386727655e-10, 3: 0.9728701375001918, 7: 0.027129862313380014}

{0: 0.0, 3: 0.9788815339915834, 7: 0.02111846600841658}

In [86]:
name = 'oct4_encumberedheuristic_oct4_encumberedheuristic_3_base_undersell_allowed-cfr_port_7_extexternal_plus_full-101'
run = EquilibriumSolverRun.objects.get(name=name, experiment__name='oct13_external_heuristics_v1')
game, final_checkpoint, policy = get_results(run, load_policy=True)
evaluation = final_checkpoint.get_modal_eval()
env_and_policy = make_env_and_policy(game, dict(final_checkpoint.equilibrium_solver_run.config))
for i, agent in enumerate(env_and_policy.agents):
    agent.policy = CFRLastIteratePolicyWrapper(policy, i)

for player in range(game.num_players()):
    env_and_policy.agents[player] = ModalAgentDecorator(env_and_policy.agents[player])
    
modal_policy = env_and_policy.make_policy()
modal_retval = nash_conv(game, modal_policy, return_only_nash_conv=False, restrict_to_heuristics=True)
display(modal_retval)

_NashConvReturn(nash_conv=20.24767626552017, player_improvements=array([18.6684278 ,  1.57924847]), br_policies=[<open_spiel.python.algorithms.best_response.BestResponsePolicy object at 0x7fb018940850>, <open_spiel.python.algorithms.best_response.BestResponsePolicy object at 0x7fb018530be0>])

# Dig into differences

In [76]:
from open_spiel.python.algorithms.exploitability import _state_values

def traverse(state, policy, cfr_policy, br_policies, br_player, follow_br=False, depth=0):
    def print_depth(s):
        if depth == 0:
            prefix = ''
        else:
            prefix = '--' * (depth-1) + '- '
        print(f'{prefix}{s}')
        
    if state.is_terminal():
        player_return = state.returns()[br_player]
        print_depth(f'Terminal state: utility = {player_return:.2f}')
        return
        
    # compute values
    player_value = _state_values(state, 2, policy)[br_player]
    br_value = br_policies[br_player].value(state)
    print_depth(f'Values: mode = {player_value:.2f}; BR = {br_value:.2f}')

    if state.is_chance_node():
        chance_outcomes = state.chance_outcomes()
        # print_depth(f'Chance node')
        for (action, action_prob) in chance_outcomes:
            print_depth(f'Chance action {action} (p={action_prob:.2f}):')
            traverse(state.child(action), policy, cfr_policy, br_policies, br_player, follow_br, depth+1)
            
    else:
        player = state.current_player()
        action_probs = policy._agents[player].policy.action_probabilities(state)
        mode_action = max(action_probs, key=action_probs.get)
        infostate_string = state.information_state_string()
        regrets, avg_policy, visits, _ = cfr_policy._infostates[infostate_string]
        avg_policy_visits = avg_policy.sum().round()
        regret_visits = visits - avg_policy_visits
        
        
        if player == br_player:
            br_action_probs = br_policies[player].action_probabilities(state)
            br_action = max(br_action_probs, key=br_action_probs.get)
        
            if br_action != mode_action:
                print_depth(f'[ BR != mode ({br_action} != {mode_action})')
                print_depth(f'[ modal value: {_state_values(state.child(br_action), 2, policy)[br_player]:5.2f}')
                print_depth(f'[ CFR stats:')
                print_depth(f'[ * regrets: {regrets.round(2)}')
                print_depth(f'[ * avg_policy: {avg_policy.round(2)}')
                print_depth(f'[ * regret visits: {regret_visits}')
                print_depth(f'[ * avg policy visits: {avg_policy_visits}')
        
        if player == br_player and follow_br:
            print_depth(f'P{player} BR action = {br_action} (policy p = {action_probs[br_action]:.2f}; {regret_visits} regret updates; regrets = {regrets.round(2)}):')       
            traverse(state.child(br_action), policy, cfr_policy, br_policies, br_player, follow_br, depth+1)
            
        else:
            print_depth(f'P{player} modal action = {mode_action} (p = {action_probs[mode_action]:.2f}; {regret_visits} regret updates; regrets = {regrets.round(2)}):')
            traverse(state.child(mode_action), policy, cfr_policy, br_policies, br_player, follow_br, depth+1)

In [77]:
name = 'oct4_encumberedheuristic_oct4_encumberedheuristic_3_base_undersell_allowed-cfr_port_7_extexternal_plus_full-101'
run = EquilibriumSolverRun.objects.get(name=name, experiment__name='oct13_external_heuristics_v1')
game, final_checkpoint, policy = get_results(run, load_policy=True)
evaluation = final_checkpoint.get_modal_eval()
env_and_policy = make_env_and_policy(game, dict(final_checkpoint.equilibrium_solver_run.config))
for player in range(game.num_players()):
    env_and_policy.agents[player].policy = policy
    env_and_policy.agents[player] = ModalAgentDecorator(env_and_policy.agents[player])
    
modal_policy = env_and_policy.make_policy()
modal_retval = nash_conv(game, modal_policy, return_only_nash_conv=False, restrict_to_heuristics=True)
display(modal_retval)

_NashConvReturn(nash_conv=17.431923076923077, player_improvements=array([17.43192308,  0.        ]), br_policies=[<open_spiel.python.algorithms.best_response.BestResponsePolicy object at 0x7f986a47b7f0>, <open_spiel.python.algorithms.best_response.BestResponsePolicy object at 0x7f98658480d0>])

In [78]:
traverse(game.new_initial_state(), modal_policy, modal_policy._agents[0].policy, modal_retval.br_policies, 0)

Values: mode = 41.73; BR = 59.16
Chance action 0 (p=0.50):
- Values: mode = 66.16; BR = 66.16
- Chance action 0 (p=0.50):
--- Values: mode = 59.92; BR = 59.92
--- P0 modal action = 3 (p = 0.97; 5211.0 regret updates; regrets = [  0.   565.46  12.2 ]):
----- Values: mode = 59.92; BR = 59.92
----- P1 modal action = 7 (p = 1.00; 5244.0 regret updates; regrets = [ 0.   33.68]):
------- Values: mode = 59.92; BR = 59.92
------- P0 modal action = 3 (p = 1.00; 5173.0 regret updates; regrets = [ 0.   36.55]):
--------- Values: mode = 59.92; BR = 59.92
--------- P1 modal action = 7 (p = 0.55; 2689.0 regret updates; regrets = [ 0.    0.   28.23]):
----------- Values: mode = 59.92; BR = 59.92
----------- P0 modal action = 3 (p = 0.99; 1476.0 regret updates; regrets = [ 0.   30.29]):
------------- Values: mode = 59.92; BR = 59.92
------------- P1 modal action = 4 (p = 1.00; 2689.0 regret updates; regrets = [ 0.   28.76  0.  ]):
--------------- Terminal state: utility = 59.92
- Chance action 1 (p=0.

In [79]:
traverse(game.new_initial_state(), modal_policy, modal_policy._agents[0].policy, modal_retval.br_policies, 0, True)

Values: mode = 41.73; BR = 59.16
Chance action 0 (p=0.50):
- Values: mode = 66.16; BR = 66.16
- Chance action 0 (p=0.50):
--- Values: mode = 59.92; BR = 59.92
--- P0 BR action = 3 (policy p = 0.97; 5211.0 regret updates; regrets = [  0.   565.46  12.2 ]):
----- Values: mode = 59.92; BR = 59.92
----- P1 modal action = 7 (p = 1.00; 5244.0 regret updates; regrets = [ 0.   33.68]):
------- Values: mode = 59.92; BR = 59.92
------- P0 BR action = 3 (policy p = 1.00; 5173.0 regret updates; regrets = [ 0.   36.55]):
--------- Values: mode = 59.92; BR = 59.92
--------- P1 modal action = 7 (p = 0.55; 2689.0 regret updates; regrets = [ 0.    0.   28.23]):
----------- Values: mode = 59.92; BR = 59.92
----------- P0 BR action = 3 (policy p = 0.99; 1476.0 regret updates; regrets = [ 0.   30.29]):
------------- Values: mode = 59.92; BR = 59.92
------------- P1 modal action = 4 (p = 1.00; 2689.0 regret updates; regrets = [ 0.   28.76  0.  ]):
--------------- Terminal state: utility = 59.92
- Chance ac

In [80]:
env_and_policy = make_env_and_policy(game, dict(final_checkpoint.equilibrium_solver_run.config))
for player in range(game.num_players()):
    env_and_policy.agents[player].policy = CFRLastIteratePolicyWrapper(policy, player)
    env_and_policy.agents[player] = ModalAgentDecorator(env_and_policy.agents[player])
    
modal_policy = env_and_policy.make_policy()
modal_retval = nash_conv(game, modal_policy, return_only_nash_conv=False, restrict_to_heuristics=True)

In [81]:
traverse(game.new_initial_state(), modal_policy, modal_policy._agents[0].policy.policy, modal_retval.br_policies, 0)

Values: mode = 41.73; BR = 59.16
Chance action 0 (p=0.50):
- Values: mode = 66.16; BR = 66.16
- Chance action 0 (p=0.50):
--- Values: mode = 59.92; BR = 59.92
--- P0 modal action = 3 (p = 0.98; 5211.0 regret updates; regrets = [  0.   565.46  12.2 ]):
----- Values: mode = 59.92; BR = 59.92
----- P1 modal action = 7 (p = 1.00; 5244.0 regret updates; regrets = [ 0.   33.68]):
------- Values: mode = 59.92; BR = 59.92
------- P0 modal action = 3 (p = 1.00; 5173.0 regret updates; regrets = [ 0.   36.55]):
--------- Values: mode = 59.92; BR = 59.92
--------- P1 modal action = 7 (p = 1.00; 2689.0 regret updates; regrets = [ 0.    0.   28.23]):
----------- Values: mode = 59.92; BR = 59.92
----------- P0 modal action = 3 (p = 1.00; 1476.0 regret updates; regrets = [ 0.   30.29]):
------------- Values: mode = 59.92; BR = 59.92
------------- P1 modal action = 4 (p = 1.00; 2689.0 regret updates; regrets = [ 0.   28.76  0.  ]):
--------------- Terminal state: utility = 59.92
- Chance action 1 (p=0.

In [82]:
traverse(game.new_initial_state(), modal_policy, modal_policy._agents[0].policy.policy, modal_retval.br_policies, 0, follow_br=True)

Values: mode = 41.73; BR = 59.16
Chance action 0 (p=0.50):
- Values: mode = 66.16; BR = 66.16
- Chance action 0 (p=0.50):
--- Values: mode = 59.92; BR = 59.92
--- P0 BR action = 3 (policy p = 0.98; 5211.0 regret updates; regrets = [  0.   565.46  12.2 ]):
----- Values: mode = 59.92; BR = 59.92
----- P1 modal action = 7 (p = 1.00; 5244.0 regret updates; regrets = [ 0.   33.68]):
------- Values: mode = 59.92; BR = 59.92
------- P0 BR action = 3 (policy p = 1.00; 5173.0 regret updates; regrets = [ 0.   36.55]):
--------- Values: mode = 59.92; BR = 59.92
--------- P1 modal action = 7 (p = 1.00; 2689.0 regret updates; regrets = [ 0.    0.   28.23]):
----------- Values: mode = 59.92; BR = 59.92
----------- P0 BR action = 3 (policy p = 1.00; 1476.0 regret updates; regrets = [ 0.   30.29]):
------------- Values: mode = 59.92; BR = 59.92
------------- P1 modal action = 4 (p = 1.00; 2689.0 regret updates; regrets = [ 0.   28.76  0.  ]):
--------------- Terminal state: utility = 59.92
- Chance ac

In [53]:
actions = [1, 1, 7, 7, 4, 7, 3, 7, 3, 7, 4]
state = game.new_initial_state()
for a in actions:
    state = state.child(a)

In [68]:
all_sor_profits = state.bidders[1].bidder.get_profits(state.sor_prices[-1])
sor_profits = np.array([all_sor_profits[a] for a in state.legal_actions()])
# Rescale sor_profits to be in -1 0
normalized_sor_profits = (sor_profits - sor_profits.min()) / (sor_profits.max() - sor_profits.min()) - 1 if sor_profits.max() != sor_profits.min() else np.zeros_like(sor_profits)
display(normalized_sor_profits * 1700)

array([-1700.       ,     0.       ,  -432.6883076])

In [61]:
state.legal_actions()

[0, 4, 7]

In [52]:
state._sor_bid_profits

defaultdict(list,
            {0: [-0.09615384615384615, 0.0, -1.0],
             1: [0.0, -0.02498265093684937]})

In [51]:
np.array(state._sor_bid_profits[0])

array([-0.09615385,  0.        , -1.        ])

In [ ]:
sor_profits = bidder.bidder.get_profits(self.sor_prices[-1])
# Rescale sor_profits to be in 0 1
normalized_sor_profits = (sor_profits - sor_profits.min()) / (sor_profits.max() - sor_profits.min()) - 1 if sor_profits.max() != sor_profits.min() else np.zeros_like(sor_profits)
self._sor_bid_profits[self._cur_player].append(1 - normalized_sor_profits[action])


In [20]:
0.55 * 0.54 * 0.69

0.20493

In [84]:
run.game.config

{'players': [{'type': [{'name': 'local',
     'prob': 0.5,
     'value': [0, 45, 58, 114, 0, 45, 58, 114],
     'budget': 114,
     'value_format': 'full'},
    {'name': 'regional',
     'prob': 0.5,
     'value': [0, 12, 15, 22, 126, 138, 141, 148],
     'budget': 148,
     'value_format': 'full'}]},
  {'type': [{'name': 'regional',
     'prob': 0.5,
     'value': [0, 16, 21, 43, 87, 104, 109, 130],
     'budget': 130,
     'value_format': 'full'},
    {'name': 'regional',
     'prob': 0.5,
     'value': [0, 15, 20, 38, 181, 197, 201, 219],
     'budget': 219,
     'value_format': 'full'}]}],
 'activity': [32, 19, 13],
 'licenses': [1, 1, 1],
 'increment': 0.3,
 'max_rounds': 10,
 'agent_memory': 10,
 'license_names': ['Ontario', 'Quebec'],
 'opening_price': [32, 19, 13],
 'heuristic_only': True,
 'fold_randomness': True,
 'undersell_policy': 'UNDERSELL_ALLOWED'}